# Laboratorio 6 
Integrantes: 
- Francis Aguilar 
- Angela Garcia 
- Cesar Lopez

In [141]:
import pandas as pd
import unicodedata
import string
import nltk
import re
from nltk.corpus import stopwords
from collections import Counter

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import networkx as nx

import community as community_louvain 
import collections

# ----- Análisis de tópicos con LDA (sklearn) -----
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation


## Descripcion general de los datos

In [142]:
import json
import pandas as pd

# cada linea es un json
with open("tioberny.txt", "r", encoding="utf-16") as f:
    data = [json.loads(line) for line in f]

# no todas las lineas tienen la misma informacion 
df = pd.DataFrame(data)


In [143]:
print("RESUMEN DEL DATAFRAME")
print("="*40)
    
    # Dimensiones\
print(f"Total de filas: {df.shape[0]}")
print(f"Total de columnas: {df.shape[1]}\n")
    



RESUMEN DEL DATAFRAME
Total de filas: 5019
Total de columnas: 32



In [144]:
df.info()
df.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5019 entries, 0 to 5018
Data columns (total 32 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   id                   5019 non-null   int64  
 1   id_str               5019 non-null   object 
 2   url                  5019 non-null   object 
 3   date                 5019 non-null   object 
 4   user                 5019 non-null   object 
 5   lang                 5019 non-null   object 
 6   rawContent           5019 non-null   object 
 7   replyCount           5019 non-null   int64  
 8   retweetCount         5019 non-null   int64  
 9   likeCount            5019 non-null   int64  
 10  quoteCount           5019 non-null   int64  
 11  bookmarkedCount      5019 non-null   int64  
 12  conversationId       5019 non-null   int64  
 13  conversationIdStr    5019 non-null   object 
 14  hashtags             5019 non-null   object 
 15  cashtags             5019 non-null   o

id                        0
id_str                    0
url                       0
date                      0
user                      0
lang                      0
rawContent                0
replyCount                0
retweetCount              0
likeCount                 0
quoteCount                0
bookmarkedCount           0
conversationId            0
conversationIdStr         0
hashtags                  0
cashtags                  0
mentionedUsers            0
links                     0
viewCount                81
retweetedTweet         5019
quotedTweet            4724
place                  4995
coordinates            5019
inReplyToTweetId        743
inReplyToTweetIdStr     743
inReplyToUser           649
source                    0
sourceUrl                 0
sourceLabel               0
media                     0
card                   4969
_type                     0
dtype: int64

In [145]:
df.head()

,id,id_str,url,date,user,lang,rawContent,replyCount,retweetCount,likeCount,...,coordinates,inReplyToTweetId,inReplyToTweetIdStr,inReplyToUser,source,sourceUrl,sourceLabel,media,card,_type
0,1834281080029110288,1834281080029110288,https://x.com/La_ReVoluZzion/status/1834281080...,2024-09-12 17:21:03+00:00,"{'id': 1435062946598694914, 'id_str': '1435062...",es,"_\nConfirmado Compañeres,\n\nEl impuesto por l...",0,0,0,...,None,NaN,None,None,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,"{'photos': [], 'videos': [], 'animated': []}",None,snscrape.modules.twitter.Tweet
1,1834252464092069901,1834252464092069901,https://x.com/XelaNewsGt/status/18342524640920...,2024-09-12 15:27:20+00:00,"{'id': 956650778634145792, 'id_str': '95665077...",es,#URGENTE Lo que los medios #faferos no informa...,12,80,142,...,None,NaN,None,None,"<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,"{'photos': [], 'videos': [{'thumbnailUrl': 'ht...",None,snscrape.modules.twitter.Tweet
2,1834280919336976681,1834280919336976681,https://x.com/M24095273/status/183428091933697...,2024-09-12 17:20:25+00:00,"{'id': 1087057038755143680, 'id_str': '1087057...",es,@IvanDuque @BArevalodeLeon Con que usaste PEGA...,0,0,0,...,None,1.834197e+18,1834197215415599186,"{'id': 77653794, 'id_str': '77653794', 'userna...","<a href=""https://mobile.twitter.com"" rel=""nofo...",https://mobile.twitter.com,Twitter Web App,"{'photos': [], 'videos': [], 'animated': []}",None,snscrape.modules.twitter.Tweet
3,1834280512933732694,1834280512933732694,https://x.com/carlosalbesc/status/183428051293...,2024-09-12 17:18:48+00:00,"{'id': 2881001877, 'id_str': '2881001877', 'ur...",es,@IvanDuque @BArevalodeLeon Entre Ellos se enti...,0,0,0,...,None,1.834197e+18,1834197215415599186,"{'id': 77653794, 'id_str': '77653794', 'userna...","<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,"{'photos': [], 'videos': [], 'animated': []}",None,snscrape.modules.twitter.Tweet
4,1834279986254987428,1834279986254987428,https://x.com/Brenda_AGN/status/18342799862549...,2024-09-12 17:16:42+00:00,"{'id': 3013862206, 'id_str': '3013862206', 'ur...",es,El presidente @BArevalodeLeon y la vicepreside...,0,0,0,...,None,NaN,None,None,"<a href=""http://twitter.com/download/android"" ...",http://twitter.com/download/android,Twitter for Android,{'photos': [{'url': 'https://pbs.twimg.com/med...,None,snscrape.modules.twitter.Tweet


In [146]:
# Identificar columnas con datos no hashables (listas o diccionarios)
columnas_no_hashable = [col for col in df.columns 
                       if df[col].apply(lambda x: isinstance(x, (list, dict))).any()]
print(columnas_no_hashable)


['user', 'hashtags', 'cashtags', 'mentionedUsers', 'links', 'quotedTweet', 'place', 'inReplyToUser', 'media', 'card']


In [147]:
# Crear un nuevo DataFrame con solo las columnas no hashables
df_no_hashable = df[columnas_no_hashable]
df_no_hashable.head()

,user,hashtags,cashtags,mentionedUsers,links,quotedTweet,place,inReplyToUser,media,card
0,"{'id': 1435062946598694914, 'id_str': '1435062...",[],[],"[{'id': 23664866, 'id_str': '23664866', 'usern...",[],"{'id': 1834252464092069901, 'id_str': '1834252...",None,None,"{'photos': [], 'videos': [], 'animated': []}",None
1,"{'id': 956650778634145792, 'id_str': '95665077...","[URGENTE, faferos, BernardoArévalo, NebajQuich...",[],[],[],None,None,None,"{'photos': [], 'videos': [{'thumbnailUrl': 'ht...",None
2,"{'id': 1087057038755143680, 'id_str': '1087057...",[],[],"[{'id': 77653794, 'id_str': '77653794', 'usern...",[],None,None,"{'id': 77653794, 'id_str': '77653794', 'userna...","{'photos': [], 'videos': [], 'animated': []}",None
3,"{'id': 2881001877, 'id_str': '2881001877', 'ur...",[],[],"[{'id': 77653794, 'id_str': '77653794', 'usern...",[],None,None,"{'id': 77653794, 'id_str': '77653794', 'userna...","{'photos': [], 'videos': [], 'animated': []}",None
4,"{'id': 3013862206, 'id_str': '3013862206', 'ur...",[],[],"[{'id': 945378369818058752, 'id_str': '9453783...",[],None,None,None,{'photos': [{'url': 'https://pbs.twimg.com/med...,None


In [148]:
# Filas duplicadas considerando solo columnas hashables
duplicados = df.drop(columns=columnas_no_hashable).duplicated()

print(f"Cantidad de filas duplicadas (ignorando columnas no hashables): {duplicados.sum()}")

Cantidad de filas duplicadas (ignorando columnas no hashables): 60


## Limpieza y preprocesamiento de datos

En base a la vista general de los datos logramos identificar que la columna de users tiene un json dentro, con más información específica sobre la información de la cuenta que está publicando. Para entender y separar mejor la información, se crea un nuevo df para visualizar la información de manera más ordenada. Adicional, en la columna user del df original, se remplaza por el nombre de usuario y se crea una columna adicional con el id_str del user llamada id_str_user

In [149]:
df_user = pd.json_normalize(df["user"])
print(df_user.head())

                    id               id_str                           url  \
0  1435062946598694914  1435062946598694914  https://x.com/La_ReVoluZzion   
1   956650778634145792   956650778634145792      https://x.com/XelaNewsGt   
2  1087057038755143680  1087057038755143680       https://x.com/M24095273   
3           2881001877           2881001877    https://x.com/carlosalbesc   
4           3013862206           3013862206      https://x.com/Brenda_AGN   

         username             displayname  \
0  La_ReVoluZzion      The_ReVoluZZzioN 🫡   
1      XelaNewsGt                XelaNews   
2       M24095273  VIVAlafuenteDEtodaVIDA   
3    carlosalbesc  Carlos Alberto Escobar   
4      Brenda_AGN             Brenda Lari   

                                      rawDescription  \
0  ¡Defensor de los defensores de los DDHH, PRO-L...   
1                          Noticias sin injerencias.   
2                                   CONCIENCIA TOTAL   
3                                         

In [ ]:
# Extraer solo el 'username' de cada diccionario
def obtener_username(user_dict):
    if isinstance(user_dict, dict):
        return user_dict.get("username")
    return None

# Extraer solo el 'id_str' de cada diccionario
def obtener_id_str(user_dict):
    if isinstance(user_dict, dict):
        return user_dict.get("id_str")
    return None

df["id_str_user"] = df["user"].apply(obtener_id_str)
df["user"] = df["user"].apply(obtener_username)

print(df[["user", "id_str_user"]].head())


             user          id_str_user
0  La_ReVoluZzion  1435062946598694914
1      XelaNewsGt   956650778634145792
2       M24095273  1087057038755143680
3    carlosalbesc           2881001877
4      Brenda_AGN           3013862206


In [ ]:
# Contar valores nulos en la columna 'user'
nulos = df["user"].isnull().sum()
print(f"Cantidad de usuarios nulos: {nulos}")

Cantidad de usuarios nulos: 0
